In [1]:
import os
import pickle
import requests
import numpy as np
import random
import re

In [2]:
x = 128
def list_to_string(a):
    a = str(a)
    return a.replace(' ', '')

def num_to_list(num):
    return [int(x) for x in str(num)]

list_x = num_to_list(x)
print(list_x)
print(list_to_string(list_x))

[1, 2, 8]
[1,2,8]


In [3]:
def get_input_string(x,y):
    x, y = str(x), str(y)
    input_str = f'Input:\n{x}+{y}\n'

    return input_str

def get_output_string(x,y):
    x, y = str(x), str(y)

    len_x, len_y = len(x), len(y)
    list_x, list_y = num_to_list(x), num_to_list(y)

    output_str = f'Target:\n<scratch>\n'

    output_str += f'{list_to_string(list_x)} has {len_x} digits.\n'
    output_str += f'{list_to_string(list_y)} has {len_y} digits.\n'

    C=0
    A=[]
    for i in range(max(len_x, len_y)):
        a = list_x[-1] if i < len_x else 0
        b = list_y[-1] if i < len_y else 0
        c = a + b + C
        
        output_str += f'{list_to_string(list_x)} + {list_to_string(list_y)} , A={list_to_string(A)} , C={C} , {a}+{b}+{C}={c} , A->{c%10} , C->{c//10}\n'

        A.insert(0, c % 10)
        C = c // 10

        list_x = list_x[:-1]
        list_y = list_y[:-1]

    output_str += f'{list_to_string(list_x)} + {list_to_string(list_y)} , A={list_to_string(A)} C={C} , END\n</scratch>\n'
    if C == 1:
        A.insert(0, 1)
    for a in A:
        output_str += f'{a} '

    return output_str[:-1]+'\n'

In [21]:
def make_addition_test_examples():
    input_file_path =  f'prompt_addition_test.txt'

    print(input_file_path)
    # if not os.path.exists(input_file_path):
    with open(input_file_path, 'w') as f:
        for a in range(1000):
            for b in range(1000):
                c = a + b
                f.write(f"${a}+{b}=\n")

def make_addition_test_subexamples(p=0.1, output=None, input=None):
    print(input, output)
    if not output is None and not input is None:
        test = input
        output_file_path = output
    else:
        test='prompt_addition_nonoverlap.txt'
        output_file_path = f'prompt_addition_test_{p}.txt'
    print(output_file_path)
    # if not os.path.exists(input_file_path):
    with open(output_file_path, 'w') as f:
        with open(test, 'r') as ft:
            data = ft.read()
            data_stripped = data.split('\n')[:-1]
            num_samples = len(data_stripped)
            subsampled = [data_stripped[i] for i in sorted(random.sample(range(len(data_stripped)), int(num_samples * p)))]
            # subsampled = random.sample(data_stripped, int(num_samples * p))
            for subsample in subsampled:
                x, y = re.split('\$|\+|=',subsample)[1:3]
                input_str = get_input_string(x,y)
                f.write(input_str)



In [ ]:
# make_addition_test_examples()

In [5]:
# create non-overlapping pairs of train/test
# input_file_path = 'add_examples_balanced.txt'
# output_file_path = 'prompt_addition_nonoverlap.txt'

# lines_to_remove = set()
# with open(input_file_path, "r") as f:
#     for line in f.readlines():
#         lines_to_remove.add(line.strip())

# with open("prompt_addition_test.txt", "r") as f:
#     with open("prompt_addition_nonoverlap.txt", "w") as f2:
#         for line in f.readlines():
#             for line_to_remove in lines_to_remove:
#                 if line_to_remove.startswith(line.strip('\n')):
#                     # print(len(lines_to_remove), line)
#                     lines_to_remove.remove(line_to_remove)
#                     break
#             else:
#                 f2.write(line)


In [22]:
make_addition_test_subexamples(p=0.0001)
make_addition_test_subexamples(p=0.001)
make_addition_test_subexamples(p=0.01)

None None
prompt_addition_test_0.0001.txt
None None
prompt_addition_test_0.001.txt
None None
prompt_addition_test_0.01.txt


In [31]:
def make_num_digit_examples(num_digit=1):
    input_file_path = f'prompt_addition_test_{num_digit}.txt'
    print(input_file_path)
    # if not os.path.exists(input_file_path):
    with open(input_file_path, 'w') as f:
        for a in range(10**num_digit):
            for b in range(10**num_digit):
                c = a + b
                f.write(f"${a}+{b}=\n")


def make_num_digit_nonoverlap_examples(num_digit=1):
    input_file_path = f'prompt_addition_test_{num_digit}_nonoverlap.txt'
    print(input_file_path)
    # if not os.path.exists(input_file_path):
    
    def get_abc(abc):
        [a,b] = abc.split('+')
        a = a[1:]
        b = b.split('=')[0]
        return a, b

    lines_to_remove = set()
    with open("add_examples_balanced.txt", "r") as f:
        for line in f.readlines():
            a, b = get_abc(line)
            if len(a) <=num_digit and len(b) <= num_digit:
                lines_to_remove.add(line.strip())
    
    print(len(lines_to_remove))

    with open(input_file_path, 'w') as f:
        for a in range(10**num_digit):
            for b in range(10**num_digit):
                addition_sample = f'${a}+{b}='
                for line_to_remove in lines_to_remove:
                    if line_to_remove.startswith(addition_sample):
                        lines_to_remove.remove(line_to_remove)
                        break
                else:
                    f.write(f'{addition_sample}\n')

In [37]:
make_num_digit_examples(num_digit=1)
make_num_digit_examples(num_digit=2)
make_num_digit_nonoverlap_examples(num_digit=2)


prompt_addition_test_1.txt
prompt_addition_test_2.txt
prompt_addition_test_2_nonoverlap.txt
1000


In [38]:
make_addition_test_subexamples(p=1, output='prompt_addition_test_1.txt', input='prompt_addition_test_1_copied.txt')
make_addition_test_subexamples(p=0.1, output='prompt_addition_test_2_0.1.txt', input='prompt_addition_test_2_nonoverlap.txt')


prompt_addition_test_1_copied.txt prompt_addition_test_1.txt
prompt_addition_test_1.txt
prompt_addition_test_2_nonoverlap.txt prompt_addition_test_2_0.1.txt
prompt_addition_test_2_0.1.txt


In [4]:
input_file_path = 'add_examples_balanced.txt'
output_file_path = 'prompt_train.txt'

with open(input_file_path, 'r') as f:
    lines = f.readlines()

with open(output_file_path, 'w') as f:
    for line in lines:
        x,y = re.split('\$|\+|=',line)[1:3]
        x, y = int(x), int(y)
        input_str = get_input_string(x,y)
        f.write(input_str)
